# Advanced Agent Design Patterns

## Organic

The **Organic Pattern** enables a natural, adaptive flow of agent interactions without relying on explicit routing rules. This allows the system to dynamically respond to the evolving needs of the conversation.

**Key Characteristics:**

- **Natural Flow:** Agent transitions occur organically, guided by the context and content of the conversation rather than predetermined paths.
- **Description-Based Routing:** The Group Chat Manager selects the next agent by evaluating agent descriptions and determining which best matches the current conversation needs.
- **Minimal Configuration:** No explicit handoff rules are required, making this the simplest pattern to implement while still supporting sophisticated routing capabilities.

This approach fosters flexibility and adaptability, ensuring that the most suitable agent is engaged at each stage of the conversation.



![alt text](../images/organic.png "Title")


## Implementation

Our implementation using AG2's Group Chat showcases the Organic Pattern with a team of specialized agents. Each agent's area of expertise is clearly defined in their description, enabling the Group Chat Manager to make intelligent, context-driven routing decisions.

**Key Implementation Details:**

- **Descriptive Agent Profiles:**  
  Each agent includes a detailed `description` field that outlines their domain expertise and when they should be activated.  
  > *Note:* If an agent's `description` is not specified, the `system_message` field will be used instead. However, it is recommended to define `description` separately for optimal use by the Group Chat Manager. See the example below for differences between `description` and `system_message`.

- **Group Chat Manager Configuration:**  
  The implementation uses the `GroupManagerTarget` for the chat's "After Work" phase, delegating all routing decisions to the Group Chat Manager. The Group Chat Manager is provided with an LLM via the `group_manager_args` parameter on the `AutoPattern`.

- **No Explicit Handoffs:**  
  Unlike other patterns, no explicit handoff rules are required. The Group Chat Manager automatically handles all agent transitions.

- **Shared Context:**  
  While shared context variables can be used to maintain state across different agents, they are not utilized for routing decisions in this example.

In [ ]:
from autogen import ConversableAgent, UserProxyAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import AutoPattern

# Setup LLM configuration
llm_config = LLMConfig(api_type="openai", model="gpt-5-mini")

In [ ]:
# Create specialized agents with descriptions that help the GROUP_MANAGER route appropriately
with llm_config:
  project_manager = ConversableAgent(
      name="project_manager",
      system_message="""You are a skilled project manager specializing in software development projects.
      You excel at creating project plans, setting milestones, managing timelines, allocating resources,
      conducting status meetings, and solving organizational problems.

      When responding to queries about project planning, timelines, resource allocation, risk management,
      or general project coordination, provide clear, structured guidance.

      You must utilize your experts: developer, qa_engineer, ui_ux_designer, and technical_writer to get the job done.
      """,
      description="""Answers questions about project planning, timelines,
      resource allocation, risk management, project coordination, team organization, and status updates.
      Call on this agent when the conversation involves planning, scheduling, task prioritization,
      or overall project management concerns.""",
  )

  developer = ConversableAgent(
      name="developer",
      system_message="""You are an expert software developer proficient in multiple programming languages
      and frameworks. You write clean, efficient code and can design robust software architectures.

      When asked for code solutions, architectural guidance, or implementation advice, provide
      practical, well-documented examples and explain your reasoning.

      You specialize in Python, JavaScript, cloud architecture, databases, and API development.
      """,
      description="""Answers questions about code implementation, programming languages,
      software architecture, technical solutions, APIs, databases, debugging, and development best practices.
      Call on this agent when the conversation involves writing or reviewing code, technical design decisions,
      or implementation approaches.""",
  )

  qa_engineer = ConversableAgent(
      name="qa_engineer",
      system_message="""You are a thorough QA engineer who specializes in software testing, quality
      assurance, and bug detection. You're skilled in creating test plans, writing test cases,
      performing manual and automated testing, and ensuring software meets quality standards.

      When addressing testing concerns, provide systematic approaches to verify functionality
      and identify potential issues.
      """,
      description="""Answers questions about testing strategies, test cases,
      quality assurance, bug detection, test automation, user acceptance testing, and software quality.
      Call on this agent when the conversation involves testing methodologies, quality concerns,
      finding bugs, or validating software functionality.""",
  )

  ui_ux_designer = ConversableAgent(
      name="ui_ux_designer",
      system_message="""You are a creative UI/UX designer with expertise in creating intuitive,
      accessible, and aesthetically pleasing user interfaces. You understand design principles,
      user research methodologies, and can create wireframes and mockups.

      When discussing design matters, focus on user-centered approaches and visual solutions
      that enhance user experience.
      """,
      description="""Answers questions about user interface design, user experience,
      visual design, wireframing, prototyping, usability, accessibility, and design systems.
      Call on this agent when the conversation involves design decisions, user interactions,
      visual elements, or user experience concerns.""",
  )

  technical_writer = ConversableAgent(
      name="technical_writer",
      system_message="""You are a skilled technical writer specializing in software documentation.
      You excel at creating clear, concise, and comprehensive documentation for various audiences,
      including user guides, API documentation, and technical specifications.

      When creating documentation, focus on clarity, completeness, and accessibility for the
      intended audience.
      """,
      description="""Answers questions about documentation, user guides,
      technical specifications, API docs, knowledge bases, and information architecture.
      Call on this agent when the conversation involves creating or improving documentation,
      explaining complex concepts, or organizing information for different audiences.""",
  )

# Create a user agent
user = UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS",
    code_execution_config=False
)


In [ ]:
# Create the agent pattern - AutoPattern uses the internal group chat manager
# to select agents automatically based on descriptions
agent_pattern = AutoPattern(
  initial_agent=project_manager,
    agents=[project_manager, developer, qa_engineer, ui_ux_designer, technical_writer],
    group_manager_args={"llm_config": llm_config},
    user_agent=user,
)

# Initiate the group chat with the pattern
result, final_context, last_agent = initiate_group_chat(
    pattern=agent_pattern,
    messages="We need to create a new web application for inventory management. Let's start with a project plan.",
    max_rounds=15,
)